In [26]:
import xml.etree.ElementTree as et
import logging
import pandas as pd
from typing import Union

# logging at level INFO
log = logging.getLogger("xml_parse")
logging.basicConfig(level=logging.INFO)

# parse the XML and get the root 
def parse_XML(xml_file):

    print(f"Dictonary for STIG_INFO from {xml_file}")
  
    # set the root node for the tree structure
    root = et.parse(xml_file).getroot()

    # call to get_stig_info and create dictionary
    si_data_dict = get_stig_info(root)

    # print(si_data_dict)
    print(get_vuln(root))

# create dictionary for SI_DATA
def get_stig_info(root) -> dict[str, Union[str, bool]]:
    stig_info_elem = root.find('.//STIG_INFO')
    si_data_dict = {}
    for si_data in stig_info_elem.findall("SI_DATA"):
        sid_name = si_data.find("SID_NAME").text 
        sid_data_el = si_data.find("SID_DATA")

        # not all the tags are the same within SI_DATA
        if sid_data_el is None:
            sid_data = True
        else:
            sid_data = sid_data_el.text

        si_data_dict[sid_name] = sid_data
    
    return si_data_dict

def get_vuln(root):
    vuln_elems = root.findall('.//VULN')
    stig_data_list = []
    for vuln_elem in vuln_elems:
        for stig_data in vuln_elem.findall("STIG_DATA"):
            vuln_attribute = stig_data.find("VULN_ATTRIBUTE").text
            attribute_data_el = stig_data.find("ATTRIBUTE_DATA")
            
            if attribute_data_el is None:
                attribute_data = True
            else:
                attribute_data = attribute_data_el.text
            stig_data_list.append(vuln_attribute)
            stig_data_list.append(attribute_data)
    return stig_data_list
    
parse_XML ("CHECKLIST_TEMPLATE_RHEL.ckl")


proxy display.', 'Vuln_Discuss', 'When X11 forwarding is enabled, there may be additional exposure to the server and client displays if the sshd proxy display is configured to listen on the wildcard address.  By default, sshd binds the forwarding server to the loopback address and sets the hostname part of the DIPSLAY environment variable to localhost.  This prevents remote hosts from connecting to the proxy display.', 'IA_Controls', None, 'Check_Content', 'Verify the SSH daemon prevents remote hosts from connecting to the proxy display.\n\nCheck the SSH X11UseLocalhost setting with the following command:\n\n# sudo grep -i x11uselocalhost /etc/ssh/sshd_config\nX11UseLocalhost yes\n\nIf the "X11UseLocalhost" keyword is set to "no", is missing, or is commented out, this is a finding.', 'Fix_Text', 'Configure the SSH daemon to prevent remote hosts from connecting to the proxy display.\n\nEdit the "/etc/ssh/sshd_config" file to uncomment or add the line for the "X11UseLocalhost" keyword an